# 데이터 불러오기

In [1]:
import sys

In [2]:
sys.path.append(r'C:/Users/user/Desktop/논문리뷰/Pattern-Exploiting-Training(PET)/code/testPET - 복사본/src')

In [3]:
import helper_functions
import pet.modeling
import pet.wrapper

In [4]:
import classification

In [5]:
import pandas as pd
import numpy as np

train, val, test, unlabeled

In [6]:
df_train_cls = pd.read_csv("C:/Users/user/Desktop/논문리뷰/Pattern-Exploiting-Training(PET)/code/testPET - 복사본/data/df_train_400_train.csv")
df_test_cls = pd.read_csv("C:/Users/user/Desktop/논문리뷰/Pattern-Exploiting-Training(PET)/code/testPET - 복사본/data/df_test.csv")
df_unlabeled_cls = pd.read_csv("C:/Users/user/Desktop/논문리뷰/Pattern-Exploiting-Training(PET)/code/testPET - 복사본/data/df_train_400_unlabeled.csv")
df_val_cls = pd.read_csv("C:/Users/user/Desktop/논문리뷰/Pattern-Exploiting-Training(PET)/code/testPET - 복사본/data/df_eval_400.csv")

In [7]:
df_train_cls = df_train_cls.astype(str)
df_test_cls = df_test_cls.astype(str)[:10]
df_unlabeled_cls = df_unlabeled_cls.astype(str)
df_val_cls = df_val_cls.astype(str)

In [8]:
label_list = np.array(['1', '2'])

In [9]:
unlabeled_probabilities = np.load('C:/Users/user/Desktop/논문리뷰/Pattern-Exploiting-Training(PET)/code/testPET - 복사본/data/classification/unlabeled_probabilities_clustering.npy')

In [10]:
train_data = df_train_cls['text']
train_labels = df_train_cls['label'].map({'1': 0, '2': 1})

val_data = df_val_cls['text']
val_labels = df_val_cls['label'].map({'1': 0, '2': 1})

test_data = df_test_cls['text']
test_labels = df_test_cls['label'].map({'1': 0, '2': 1})

unlabeled_data = df_unlabeled_cls['text']
unlabeled_labels = pd.Series(label_list[np.argmax(unlabeled_probabilities, axis=1)]).map({'1': 0, '2': 1})

데이터 합치기(train_data + unlabeled_data / train_labels + unlabeled_labels)

In [ ]:
train_data = pd.concat([train_data, unlabeled_data], axis=0).reset_index(drop=True)
train_labels = pd.concat([train_labels, unlabeled_labels], axis=0).reset_index(drop=True)

# Classification_just train_data

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [12]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')
config = RobertaConfig.from_pretrained('roberta-large')
config.num_labels = 2
model = RobertaForSequenceClassification(config)

In [13]:
class YelpPolarityDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text = str(self.texts.iloc[idx])
        label = int(self.labels.iloc[idx])

        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [14]:
batch_size = 8
max_length = 256

In [15]:
train_dataset = YelpPolarityDataset(train_data, train_labels, tokenizer, max_length)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = YelpPolarityDataset(val_data, val_labels, tokenizer, max_length)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

test_dataset = YelpPolarityDataset(test_data, test_labels, tokenizer, max_length)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
         

In [17]:
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    all_train_losses = []
    
    for epoch in range(num_epochs+1):
        train_losses = []
        
        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            
            train_losses.append(loss.item())
        
        average_train_loss = sum(train_losses) / len(train_losses)
        all_train_losses.append(average_train_loss)
        print(f"Epoch {epoch + 1}/{num_epochs} - Average Train Loss: {average_train_loss:.4f}")
    
        train_loss[epoch] = average_train_loss
        
        
    return all_train_losses
        

In [18]:
def validate_model(model, val_loader, criterion):
    model.eval()
    all_val_losses = []
    all_val_accuracy = []
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        epoch_losses = []
        
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            epoch_losses.append(loss.item())

            logits = outputs.logits
            _, predicted_labels = torch.max(logits, 1)
            total_correct += (predicted_labels == labels).sum().item()
            total_samples += labels.size(0)

    average_loss = sum(epoch_losses) / len(epoch_losses)
    all_val_losses.append(average_loss)
    accuracy = total_correct / total_samples
    all_val_accuracy.append(accuracy)
    
    val_loss[epoch] = average_loss
    val_accuracy[epoch] = accuracy
    
    print(f"Epoch {epoch + 1}/{num_epochs} - Val Loss: {average_loss:.4f} - Accuracy: {accuracy}")

    return all_val_losses, all_val_accuracy

In [19]:
def calculate_accuracy(model, test_loader):
    model.eval()
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            logits = outputs.logits
            _, predicted_labels = torch.max(logits, 1)

            total_correct += (predicted_labels == labels).sum().item()
            total_samples += labels.size(0)

    accuracy = total_correct / total_samples
    print(f"Accuracy: {accuracy:.4f}")
    
    return accuracy

In [21]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
num_epochs = 1
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 1.0, gamma=0.95)

In [ ]:
epochs=1
train_loss = {}
val_loss = {}
val_accuracy = {}

for epoch in tqdm(range(1, epochs+1)):
  
  train_model(model, train_loader, criterion, optimizer, num_epochs)
  validate_model(model, val_loader, criterion)
  
  torch.save(model, f'./trained_classifier/clustering/checkpoint_{epoch}ep.pt')

  scheduler.step()

In [40]:
train_loss

{}

In [83]:
val_loss

{1: 0.7170287455831256, 2: 0.6895235776901245}

In [84]:
val_accuracy

{1: 0.5, 2: 0.5}

In [106]:
test_accuracy = calculate_accuracy(model, test_loader)

Test Accuracy: 0.5000


# (논문) classification

In [33]:
train_and_score_logistic_regression_model(
    df_train['text'], df_train['label'], df_test['text'], df_test['label']
)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

unlabel data 추가하여 학습한 모델

In [17]:
helper_functions.train_and_score_logistic_regression_model(
    df_train['text'],
    df_train['label'],
    df_test['text'],
    df_test['label'],
    unlabeled_X_train=df_unlabeled['text'],
    unlabeled_probabilities=unlabeled_probabilities,
    label_list=label_list)

0.8277368421052631